In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("/mnt/d1/KEX/.myenv/GPT-sw3-126m-BaseLargeTunedHyp_3Epoch_myown")
model = AutoModelForCausalLM.from_pretrained("/mnt/d1/KEX/.myenv/GPT-sw3-126m-BaseLargeTunedHyp_3Epoch_myown")
model.eval()
model.to(device)

#iput_fran_FAKS är mätvärdenra i den årdning som står i excel, utan läkarens svar såklart.
#Det bör se ut så här: <|endoftext|><s>User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: ['100 - 176', 1.2, 76, 0.0, nan, nan, 'Nedsatt', 'Tricuspid', nan, nan, nan, 26.0, '27 - 39', 13.9, 24.0, 12.8, 1.8, nan, nan, nan, nan, '10,3 - 10,3', 35.0, '25 - 38', 18.7, nan, 13.0, nan, 'Inga vegetationsmisstänkta förändringar.', nan, nan, nan, nan, nan, nan, nan, 'Normal', 'ala', nan, 23.5, 1.87, nan, nan, nan, nan, nan, nan, 80.0, nan, nan, nan, '142 - 258', 0.8, '0,96 - 1,32', 0.96, nan, '53 - 73', nan, '54 - 74', nan, 59.0, nan, nan, 'Förtjockad', nan, nan, nan, nan, 34.0, 'Säng', nan, nan, nan, 80.0, 'Kvinna', 175.0, 26.0, 13.9, '6 - 11', nan, 'Nedsatt', 19.0, '22 - 40', 'Tricuspidalisinsufficiensens Vmax svåravgränsad.  ', nan, nan, 21.0, nan, '73 - 101', 16.0, 25.0, 13.36898395721925, nan, nan, nan, nan, nan, nan, 33.0, 17.65, '23 - 31\t32 - 34\t35 - 37\t ≥ 37', nan, nan, '29 - 61', nan, 55.0, nan, nan, '6 - 22', 190.0, 162, 102.0, 95, '43 - 95\t96 - 108\t109 - 121\t ≥ 122', nan, 0.9, nan, nan, 15.0, 0.0, nan, nan, nan, nan, nan, 'Inga vegetationsmisstänkta förändringar.', nan, nan, nan, nan, nan, 69.0, 'ja', nan, 18.0, 15.0, nan, 12.0, nan, 'Ingen kompression av högersidiga hjärtrum.', nan, nan, nan, 16.0, 1.0, 0.84, 16.0, nan, nan, nan, 1.5, nan, 3.2, nan, nan, nan, nan, 10.0, 1.6, 'Lätt ökad ekotäthet, men inga vegetationsmisstänkta förändringar.', nan, nan, 8.0, nan, nan, nan, 'Vegetationer? Tecken på endokardit?', "Kvinna med tidigare myelom, stamcelltransplantation '00, amyloidos med njurpåverkan, och restriktiv kardiomyopati, LE 2007, PH. Har utvecklat staphylokock sepsis med ingångsport PICC-lines instickstället. Vid hjärt-auskultation syst. blåsljud med PM I2 sin. Tacksam för TEE för att utesluta endokardit. Mvh Ioanna Kotortsi, leg läkare Lung Allergi Klinik", 44.0, 23.52941176470588, 'Referens ej tillgänglig', nan, nan, nan, nan, nan, 0.9090909090909091, 'Regelbunden', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '5 - 12', nan, nan, nan, nan, 130.0, 15.0, nan, 16.0, 0.06, nan, nan, 'Lokal svalgbedövning med Xylocainspray. \nKommentar TEE\nEfter svalganestesin börjar pat må illa, kräks lite och harklar upp blodstrimmigt slem. Mtp patientens allmäntillstånd, och eftersom den transthorakala undersökningen inte ingav misstanke om någon klaffvegetation, avstås därför från transesofagal undersökning idag.', nan, nan, nan, nan, nan, nan, nan, 'Ej genomförbart', nan, nan, nan, nan, 1.0, 61.0, '0 - 32,1', nan, nan, nan, nan, nan, nan, 3.9, '0 - 2,8', nan, nan, 'Inga vegetationsmisstänkta förändringar.', nan, nan, nan, 'Systodiastolisk septumavplaning. Fusionerad mitralis E- och A-våg.', nan, 37.0, 19.8, 72.0, nan, nan, '29 - 61', nan, nan, nan, '6 - 22', nan, 5075, nan, nan, '29 - 61', nan, nan, nan, nan, nan, nan, '50 - 75', nan, nan, '6 - 22', nan, nan, '26 - 58', nan, '52 - 72', nan, '50 - 75', nan, nan, '8 - 24', 44.13315789473684, 18.12106382978724, 23.0, 12.0]<s>Bot:

input_text = f"<|endoftext|><s>User: Skriv en patientjornal efter en ultraljudsundersökning utifrån dessa värden: {input_fran_FAKS}<s>Bot:"
prompt = input_text.strip()


token_count = len(tokenizer.encode_plus(prompt)["input_ids"])
max_token_count = 2048 - 140

if token_count > max_token_count:
    # Hitta indexet där de tre sista elementen börjar
    end_index = len(prompt) - 7
    end_seq = prompt[end_index:]
    tokens = tokenizer.encode_plus(prompt[:end_index])["input_ids"]
    tokens = tokens[:max_token_count]
    prompt = (tokenizer.decode(tokens) + (end_seq)).strip()
    

generator = pipeline('text-generation', tokenizer=tokenizer, model=model, device=device)
generated = generator(prompt, max_new_tokens=140, do_sample=True, temperature=0.6, top_p=1)[0]["generated_text"]
print(generated)